In [12]:
import torch
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
import torch.utils.data
from efficientnet_pytorch import EfficientNet
import torch.nn as nn
import torch.optim
from torch.autograd import Variable
import os
import pickle
import sys
import tarfile
import numpy as np
from PIL import Image
from six.moves.urllib.request import urlretrieve
import urllib.error
import torchtext
import argparse
import math
import time
import scipy.misc
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib
import matplotlib.pyplot as plt

In [13]:
cifar100_stats = ((0.507, 0.487, 0.441), (0.267, 0.256, 0.276))
cifar10_stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
transform = transforms.Compose(
[transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(),
 transforms.ToTensor(),
     transforms.Normalize(cifar100_stats[0], cifar10_stats[1])])

test_transform = transforms.Compose(
[transforms.ToTensor(),
     transforms.Normalize(cifar100_stats[0], cifar10_stats[1])])
batch_size = 256

#val_size = 1500

dataset = torchvision.datasets.CIFAR10(root='C:\\Users\Cena\PycharmProjects\csc413 final', train=True,
                                        download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='C:\\Users\Cena\PycharmProjects\csc413 final', train=False,
                                       download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=200,
                                         shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [15]:
PATH = 'C:\\Users\Cena\PycharmProjects\csc413 final\efficientnetb0_0_cifar100.pt'
pre_trained = True
if pre_trained:
    net = EfficientNet.from_pretrained('efficientnet-b2')
else:
    net = EfficientNet.from_name("efficientnet-b2 ")


#uncomment these block to load trained model
load_model = False
if load_model:
    state_dict = torch.load(PATH)
    # create new OrderedDict that does not contain `module.`
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] # remove `module.`
        new_state_dict[name] = v
    # load params
    net.load_state_dict(new_state_dict)
net = net.cuda()

net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
#cudnn.benchmark = True


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b2-8bb594d6.pth" to C:\Users\Cena/.cache\torch\hub\checkpoints\efficientnet-b2-8bb594d6.pth



Loaded pretrained weights for efficientnet-b2


In [16]:
num_epochs = 200
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

loss_list = []
test_acc_list = []
lrSchduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    test_loss = 0.0
    net.train()
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        inputs = inputs.cuda()
        labels = labels.cuda()

        # wrap them in Variable
        #inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        # print statistics
        running_loss += loss.data.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Normalizing the loss by the total number of train batches
    running_loss /= len(trainloader)

    # Calculate training accuracy of the existing model

    train_accuracy = correct / total
    loss_list.append(running_loss)

    test_correct = 0
    test_total = 0

    net.eval()
    with torch.no_grad():
        for data in testloader:
            test_images, test_labels = data
            test_inputs = test_images.cuda()
            test_labels = test_labels.cuda()
            test_outputs = net(test_inputs)
            test_loss = criterion(test_outputs, test_labels)
            _, predicted = torch.max(test_outputs.data, 1)
            test_total += test_labels.size(0)
            test_correct += (predicted == test_labels).sum().item()
    test_acc = test_correct / test_total
    test_acc_list.append(test_acc)

    print("Iteration: {0} | Loss: {1} | Training accuracy: {2} | test loss: {3} | test_acc: {4}".format(epoch+1,
                                                                                              running_loss,
                                                                                              train_accuracy, test_loss,
                                                                                                   test_acc
                                                                                            ))
    lrSchduler.step()
save_model = False
if save_model:
    torch.save(net.state_dict(), PATH)
print('==> Finished Training ...')


Iteration: 1 | Loss: 3.3150501944580855 | Training accuracy: 0.12976 | test loss: 412.65484619140625 | test_acc: 0.1
Iteration: 2 | Loss: 2.135742258660647 | Training accuracy: 0.20512 | test loss: 2.2869760990142822 | test_acc: 0.1004
Iteration: 3 | Loss: 1.9428852097112306 | Training accuracy: 0.26648 | test loss: 2.463998556137085 | test_acc: 0.1024
Iteration: 4 | Loss: 1.8035004941784605 | Training accuracy: 0.32034 | test loss: 2.388169288635254 | test_acc: 0.134
Iteration: 5 | Loss: 1.7187806428695211 | Training accuracy: 0.35942 | test loss: 2.5920629501342773 | test_acc: 0.1
Iteration: 6 | Loss: 1.647651992282089 | Training accuracy: 0.38316 | test loss: 2.5447309017181396 | test_acc: 0.1388
Iteration: 7 | Loss: 1.553493862857624 | Training accuracy: 0.4252 | test loss: 2.603900194168091 | test_acc: 0.1089
Iteration: 8 | Loss: 1.4763317752857597 | Training accuracy: 0.45812 | test loss: 2.533982753753662 | test_acc: 0.139
Iteration: 9 | Loss: 1.411367590330085 | Training accura

In [17]:
"""if model_version == 0:
    PATH = 'C:\\Users\Cena\PycharmProjects\csc413 final\efficientnetb0.pt'
    model = EfficientNetB0()
else:
    PATH = 'C:\\Users\Cena\PycharmProjects\csc413 final\efficientnetb0_0.pt'
    model = EfficientNet.from_name('efficientnet-b0')

state_dict = torch.load(PATH)
# create new OrderedDict that does not contain `module.`
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
# load params
model.load_state_dict(new_state_dict)
net = model.cuda()
net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))"""

"""correct = 0
total = 0
with torch.no_grad():
    net.eval()
    for data in testloader:
        images, labels = data
        inputs = images.cuda()
        labels = labels.cuda()
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images:',
    correct / total)"""

best_test_acc = max(test_acc_list)
index = test_acc_list.index(max(test_acc_list)) + 1
print("Best test acc during training: ", best_test_acc, " epoch: ", index)

print("done")

Best test acc during training:  0.8802  epoch:  191
done
